# Import Libraries
---

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.data import Dataset

import tensorflow_datasets as tfds

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Device selection in TensorFlow
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    device = "GPU"
    print("Using device: GPU")
else:
    device = "CPU"
    print("Using device: CPU")

In [ ]:
# Load MNIST from TensorFlow Datasets
ds, ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

train_ds, test_ds = ds

# Normalize images and batch datasets
def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Shape: (28, 28, 1)
    return image, label

batch_size = 128

train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.cache().shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

# Count samples
num_train_samples = ds_info.splits['train'].num_examples
num_test_samples = ds_info.splits['test'].num_examples

print(f"Train samples: {num_train_samples}, Test samples: {num_test_samples}")

# Data Explainantion

---

In [ ]:
# Load the dataset as before
ds, ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)
train_ds, test_ds = ds

# 1. Number of classes
num_classes = ds_info.features['label'].num_classes
print(f"Number of classes in the dataset: {num_classes}")

# 2. Class Distribution and Imbalance Check
# Extract all labels from the train dataset
train_labels = []
for _, label in tfds.as_numpy(train_ds):
    train_labels.append(label)
train_labels = np.array(train_labels)

class_counts = np.bincount(train_labels)
class_labels = np.arange(num_classes)

In [ ]:
plt.figure(figsize=(8, 5.5))
sns.barplot(x=class_labels, y=class_counts, palette="viridis", dodge=False)
plt.title("Class Distribution in MNIST Training Set")
plt.xlabel("Digit Class")
plt.ylabel("Number of Samples")
plt.xticks(class_labels)
for i, v in enumerate(class_counts):
    plt.text(i, v, str(v), color='black', va="bottom")
plt.savefig("class_distribution.png")
plt.show()

In [ ]:
# Load train dataset as before
train_ds = tfds.load('mnist', split='train', as_supervised=True)
num_classes = 10

# Collect one sample image for each class
sample_images = [None] * num_classes

for image, label in tfds.as_numpy(train_ds):
    if sample_images[label] is None:
        sample_images[label] = image
    if all(img is not None for img in sample_images):
        break

plt.figure(figsize=(12, 6))
for i in range(num_classes):
    plt.subplot(2, 5, i + 1)
    plt.imshow(np.squeeze(sample_images[i]), cmap="gray")
    plt.title(f"Class {i}")
    plt.axis("off")

plt.suptitle("Sample Images from Each Class in MNIST Training Set")
plt.savefig("sample_images.png")
plt.show()

Initial Clustering training Images
---

In [ ]:
import tensorflow_datasets as tfds
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

# Load train images (as before)
train_ds = tfds.load('mnist', split='train', as_supervised=True)

# Collect all images (as numpy, normalized)
raw_train_images = []
for image, _ in tfds.as_numpy(train_ds):
    # image shape (28, 28, 1)
    raw_train_images.append(image.astype(np.float32) / 255.0)

raw_train_images = np.stack(raw_train_images)  # shape (N, 28, 28, 1)
raw_train_images_flat = raw_train_images.reshape(-1, 28*28)  # (N, 784)

# KMeans clustering
kmeans_init = KMeans(n_clusters=10, n_init=20, random_state=42)
initial_kmeans_labels = kmeans_init.fit_predict(raw_train_images_flat)

print("Initial clustering completed.")

# t-SNE visualization (subsample 1000 images for speed)
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=500)
raw_images_2d = tsne.fit_transform(raw_train_images_flat[:1000])

plt.figure(figsize=(10, 7))
sns.scatterplot(x=raw_images_2d[:, 0], y=raw_images_2d[:, 1], hue=initial_kmeans_labels[:1000], palette='tab10', s=15)
plt.title("Initial KMeans Clustering on Raw MNIST Training Images")
plt.savefig('initial_clustering_raw.png')
plt.show()
plt.close()

print("Visualization completed. Plot saved as 'initial_clustering_raw.png'.")

# Define Model
---

In [ ]:
def preprocess_for_resnet(batch):
    # batch shape: (batch_size, 28, 28, 1)
    # Resize
    batch = tf.image.resize(batch, [224, 224], method='bilinear')
    # Repeat channels: from (batch_size, 224, 224, 1) to (batch_size, 224, 224, 3)
    batch = tf.image.grayscale_to_rgb(batch)
    return batch

# Get one batch from the training dataset iterator
images, labels = next(iter(train_ds))

print("Original batch shape:", images.shape)

# Apply preprocessing for ResNet
inputs = preprocess_for_resnet(images)
print("After preprocess_for_resnet:", inputs.shape)

In [1]:
class ResNetAutoencoder(tf.keras.Model):
    def __init__(self, base_model="resnet50", pretrained=True):
        super().__init__()
        # 1) Load any Keras ResNet and strip off avgpool+fc
        if base_model == "resnet18":
            raise NotImplementedError("ResNet18 is not available in tf.keras.applications, use resnet50 or resnet101, etc.")
        keras_resnet = getattr(tf.keras.applications, base_model)(
            include_top=False,
            weights='imagenet' if pretrained else None,
            input_shape=(224, 224, 3)
        )
        self.encoder = keras_resnet

        # 2) Figure out how many channels it outputs
        dummy = tf.zeros((1, 224, 224, 3))
        feat = self.encoder(dummy)
        feat_dim = feat.shape[-1]      # e.g. 2048 for ResNet50
        self._feat_dim = feat_dim

        # 3) Decoder: (batch, 7, 7, feat_dim) → (batch, 28, 28, 1)
        self.decoder = models.Sequential([
            layers.Conv2DTranspose(feat_dim // 2, kernel_size=3, strides=2, padding="same", output_padding=1), # (14, 14, feat_dim//2)
            layers.ReLU(),
            layers.Conv2DTranspose(feat_dim // 4, kernel_size=3, strides=2, padding="same", output_padding=1), # (28, 28, feat_dim//4)
            layers.ReLU(),
            layers.Conv2D(feat_dim // 4, 64, kernel_size=3, padding="same"),
            layers.ReLU(),
            layers.Conv2D(64, 1, kernel_size=3, padding="same"),
            layers.Activation('sigmoid')
        ])

    def call(self, x):
        z = self.encoder(x)  # (batch, 7, 7, feat_dim)
        z_flat = tf.reduce_mean(z, axis=[1, 2])  # global average pooling (batch, feat_dim)
        recon = self.decoder(z)  # (batch, 28, 28, 1)
        return recon, z_flat

Pre-train Autoencoder
---

In [ ]:
# Instantiate the model
autoencoder = ResNetAutoencoder(base_model="ResNet50")  # Note: use "ResNet50" (capitalization matches tf.keras.applications)

# Build model once to initialize variables (dummy input)
dummy_input = tf.zeros((1, 224, 224, 3))
recon, z_flat = autoencoder(dummy_input)

# Loss function (Mean Squared Error)
loss_fn = tf.keras.losses.MeanSquaredError()

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

print("Encoder output channels:", autoencoder._feat_dim)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    # ——— Training ———
    running_train_loss = 0.0
    num_train_samples = 0

    for images, _ in train_ds:  # train_ds is batched
        # Preprocess for ResNet
        inputs = preprocess_for_resnet(images)
        # Forward and backward pass
        with tf.GradientTape() as tape:
            outputs, _ = autoencoder(inputs, training=True)
            loss = loss_fn(images, outputs)
        grads = tape.gradient(loss, autoencoder.trainable_variables)
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_variables))

        batch_size = tf.shape(images)[0].numpy()
        running_train_loss += loss.numpy() * batch_size
        num_train_samples += batch_size

    avg_train_loss = running_train_loss / num_train_samples

    # ——— Validation ———
    running_val_loss = 0.0
    num_val_samples = 0

    for images, _ in test_ds:  # test_ds is batched
        inputs = preprocess_for_resnet(images)
        outputs, _ = autoencoder(inputs, training=False)
        loss = loss_fn(images, outputs)

        batch_size = tf.shape(images)[0].numpy()
        running_val_loss += loss.numpy() * batch_size
        num_val_samples += batch_size

    avg_val_loss = running_val_loss / num_val_samples

    print(f"Epoch [{epoch+1}/{num_epochs}]  "
          f"Train Loss: {avg_train_loss:.4f}  "
          f"Val Loss:   {avg_val_loss:.4f}")

Define SHA-12 & Loss Functions
---

In [ ]:
class DEC(tf.keras.Model):
    def __init__(self, autoencoder, n_clusters=10, latent_dim=64):
        super().__init__()
        self.autoencoder = autoencoder
        self.n_clusters = n_clusters
        self.latent_dim = latent_dim
        # Cluster centers: shape (n_clusters, latent_dim)
        self.cluster_centers = tf.Variable(
            tf.random.normal([n_clusters, latent_dim]), trainable=True, name="cluster_centers"
        )

    def call(self, x):
        _, z = self.autoencoder(x)  # z: (batch, latent_dim)
        q = self.soft_assign(z)
        return z, q

    def soft_assign(self, z):
        # z: (batch, latent_dim), cluster_centers: (n_clusters, latent_dim)
        # Expand z to (batch, 1, latent_dim) and cluster_centers to (1, n_clusters, latent_dim)
        expanded_z = tf.expand_dims(z, axis=1)
        expanded_centers = tf.expand_dims(self.cluster_centers, axis=0)
        dist = tf.reduce_sum(tf.square(expanded_z - expanded_centers), axis=2)  # (batch, n_clusters)
        q = 1.0 / (1.0 + dist)
        q = q ** ((1 + 1) / 2)
        q = q / tf.reduce_sum(q, axis=1, keepdims=True)
        return q

def target_distribution(q):
    weight = q ** 2 / tf.reduce_sum(q, axis=0)
    # Transpose, normalize, then transpose back (for batch compatibility)
    return tf.transpose(tf.transpose(weight) / tf.reduce_sum(weight, axis=1))

def kl_divergence(p, q):
    return tf.reduce_mean(tf.reduce_sum(p * tf.math.log((p + 1e-6) / (q + 1e-6)), axis=1))

Initialize SHA-12 with KMeans
---

In [ ]:
# 1) Freeze decoder
autoencoder.decoder.trainable = False

# 2) Gather latents (with proper preprocessing!)
latent_list = []
for images, _ in train_ds:  # train_ds is batched
    inputs = preprocess_for_resnet(images)
    _, z = autoencoder(inputs, training=False)  # (batch, latent_dim)
    latent_list.append(z.numpy())
latent_features = np.concatenate(latent_list, axis=0)

# KMeans init
kmeans = KMeans(n_clusters=10, n_init=20, random_state=42)
kmeans.fit(latent_features)

# DEC model
dec_model = DEC(autoencoder, n_clusters=10, latent_dim=latent_features.shape[1])

# Set cluster centers to KMeans result
dec_model.cluster_centers.assign(kmeans.cluster_centers_)

Fine-Tune Using Clustering Loss
---

In [ ]:
dec_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
num_dec_epochs = 10

for epoch in range(num_dec_epochs):
    # ——— Training Phase ———
    train_kl = 0.0
    num_train_samples = 0

    for images, _ in train_ds:  # train_ds is batched
        inputs = preprocess_for_resnet(images)
        with tf.GradientTape() as tape:
            z, q = dec_model(inputs, training=True)
            p = target_distribution(tf.stop_gradient(q))
            loss = kl_divergence(p, q)
        grads = tape.gradient(loss, dec_model.trainable_variables)
        dec_optimizer.apply_gradients(zip(grads, dec_model.trainable_variables))

        batch_size = tf.shape(images)[0].numpy()
        train_kl += loss.numpy() * batch_size
        num_train_samples += batch_size

    avg_train_kl = train_kl / num_train_samples

    # ——— Validation Phase ———
    val_kl = 0.0
    num_val_samples = 0
    for images, _ in test_ds:  # test_ds is batched
        inputs = preprocess_for_resnet(images)
        z, q = dec_model(inputs, training=False)
        p = target_distribution(q)
        loss = kl_divergence(p, q)
        batch_size = tf.shape(images)[0].numpy()
        val_kl += loss.numpy() * batch_size
        num_val_samples += batch_size

    avg_val_kl = val_kl / num_val_samples

    print(f"DEC Epoch [{epoch+1}/{num_dec_epochs}]  "
          f"Train KL: {avg_train_kl:.6f}  "
          f"Val KL:   {avg_val_kl:.6f}")

dec_val_kl = avg_val_kl

Evaluate Clustering with Metrics
---

In [ ]:
# Collect all latents and labels
all_latents = []
all_labels = []

for images, labels in test_ds:  # test_ds is your batched tf.data.Dataset
    inputs = preprocess_for_resnet(images)
    _, z = dec_model.autoencoder(inputs, training=False)
    all_latents.append(z.numpy())
    all_labels.append(labels.numpy())

all_latents = np.vstack(all_latents)
all_labels = np.hstack(all_labels)

# KMeans clustering
kmeans_final = KMeans(n_clusters=10, random_state=42)
cluster_labels = kmeans_final.fit_predict(all_latents)

# Metrics
sil_score = silhouette_score(all_latents, cluster_labels)
db_score = davies_bouldin_score(all_latents, cluster_labels)
ch_score = calinski_harabasz_score(all_latents, cluster_labels)

print(f"Silhouette Score:        {sil_score:.4f}")
print(f"Davies-Bouldin Index:    {db_score:.4f}")
print(f"Calinski-Harabasz Index: {ch_score:.4f}")
print(f"KL: {dec_val_kl:.4f}")

Visualize Clusters with t-SNE
---

In [ ]:
# Assuming all_latents is (N, latent_dim) numpy array and cluster_labels is (N,) numpy array or list
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=500)
latents_2d = tsne.fit_transform(all_latents)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=latents_2d[:, 0], y=latents_2d[:, 1], hue=cluster_labels, palette='tab10', legend='full', s=15)
plt.title("t-SNE Visualization ")
plt.show()

SOM Processing
---

In [ ]:
class SOM:
    def __init__(self, map_size, input_dim, learning_rate=0.5, sigma=None):
        self.map_size = map_size
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.sigma = sigma if sigma is not None else max(map_size) / 2

        # Initialize weights randomly and normalize
        weights = tf.random.normal((map_size[0], map_size[1], input_dim))
        weights = weights / tf.norm(weights, axis=2, keepdims=True)
        self.weights = tf.Variable(weights, trainable=False, dtype=tf.float32)

        # Grid coordinates
        x, y = tf.meshgrid(tf.range(map_size[0]), tf.range(map_size[1]), indexing='ij')
        self.grid = tf.stack([x, y], axis=-1)
        self.grid = tf.cast(self.grid, tf.float32)

    def get_bmu(self, x):
        # x: (batch_size, input_dim)
        flat_weights = tf.reshape(self.weights, [-1, self.input_dim])  # (M*N, input_dim)
        # Compute pairwise Euclidean distances: (batch_size, M*N)
        dists = tf.norm(tf.expand_dims(x, 1) - flat_weights, axis=2)
        bmu_indices = tf.argmin(dists, axis=1)  # (batch_size,)
        bmu_coords = tf.stack([bmu_indices // self.map_size[1], bmu_indices % self.map_size[1]], axis=1)
        return bmu_coords

    def update(self, x, t, max_iter):
        lr = self.learning_rate * np.exp(-t / max_iter)
        sigma_t = self.sigma * np.exp(-t / max_iter)
        bmu_coords = self.get_bmu(x)  # (batch_size, 2)

        for i in range(x.shape[0]):
            # Calculate distance to BMU on the grid
            dist_to_bmu = tf.norm(self.grid - tf.cast(bmu_coords[i], tf.float32), axis=2)  # (map_x, map_y)
            influence = tf.exp(-tf.square(dist_to_bmu) / (2 * sigma_t**2))  # (map_x, map_y)
            diff = x[i] - self.weights  # (map_x, map_y, input_dim)
            delta = lr * tf.expand_dims(influence, -1) * diff  # (map_x, map_y, input_dim)
            self.weights.assign_add(delta)

    def predict(self, x):
        flat_weights = tf.reshape(self.weights, [-1, self.input_dim])  # (M*N, input_dim)
        dists = tf.norm(tf.expand_dims(x, 1) - flat_weights, axis=2)  # (batch_size, M*N)
        bmu_indices = tf.argmin(dists, axis=1)
        return bmu_indices.numpy()

Extract Features
---

In [ ]:
latent_features = []
test_latents = []
test_labels = []

# Gather latent features from training data
for images, _ in train_ds:
    inputs = preprocess_for_resnet(images)
    _, z = model(inputs, training=False)
    latent_features.append(z.numpy())

# Gather latent features and labels from test data
for images, labels in test_ds:
    inputs = preprocess_for_resnet(images)
    _, z = model(inputs, training=False)
    test_latents.append(z.numpy())
    test_labels.append(labels.numpy())

latent_features = np.vstack(latent_features)
test_latents = np.vstack(test_latents)
test_labels = np.hstack(test_labels)

latent_dim = latent_features.shape[1]

In [ ]:
# Initial KMeans clustering (for DEC)
kmeans_init = KMeans(n_clusters=10, n_init=20, random_state=42)
initial_kmeans_labels = kmeans_init.fit_predict(latent_features)

# Initial SOM clustering
som = SOM(map_size=(10, 10), input_dim=latent_dim)
initial_som_weights = som.weights.numpy().reshape(-1, latent_dim)
kmeans_som_init = KMeans(n_clusters=10, random_state=42)
initial_som_labels = kmeans_som_init.fit_predict(initial_som_weights)

# Visualize initial clustering
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=500)
latents_2d = tsne.fit_transform(latent_features[:1000])  # Subsample for visualization
som_weights_2d = tsne.fit_transform(initial_som_weights)

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
sns.scatterplot(x=latents_2d[:, 0], y=latents_2d[:, 1], hue=initial_kmeans_labels[:1000], palette='tab10', s=15)
plt.title("Initial KMeans Clustering (DEC)")
plt.subplot(1, 2, 2)
sns.scatterplot(x=som_weights_2d[:, 0], y=som_weights_2d[:, 1], hue=initial_som_labels, palette='tab10', s=15)
plt.title("Initial SOM Weights Clustering")
plt.savefig('initial SOM clustering.png')
plt.show()
plt.close()

Train SOM
---

In [ ]:
import numpy as np

som = SOM(map_size=(2, 5), input_dim=latent_dim)
max_iter = 1000

latent_tensor = tf.convert_to_tensor(latent_features, dtype=tf.float32)

for t in range(max_iter):
    indices = np.random.permutation(latent_features.shape[0])[:128]
    batch = tf.gather(latent_tensor, indices)
    som.update(batch, t, max_iter)
    if (t + 1) % 100 == 0:
        print(f"SOM Iteration [{t+1}/{max_iter}]")

In [ ]:
latent_tensor = tf.convert_to_tensor(test_latents, dtype=tf.float32)

# Compute distances between test latents and all SOM weights (flattened)
flat_weights = tf.reshape(som.weights, [-1, som.input_dim])  # (num_nodes, latent_dim)
dists = tf.norm(tf.expand_dims(latent_tensor, 1) - flat_weights, axis=2)  # (num_samples, num_nodes)
bmu_dists = tf.reduce_min(dists, axis=1)  # (num_samples,)

quant_error_som = tf.reduce_mean(bmu_dists).numpy()

print(f"SOM Quantization Error: {quant_error_som:.6f}")

In [ ]:
def som_detailed_summary(som):
    # Compute sizes
    rows, cols = som.map_size
    n_neurons = rows * cols
    w_shape = som.weights.shape          # (rows, cols, input_dim)
    g_shape = som.grid.shape             # (rows, cols, 2)
    # Parameter counts
    total_weights = np.prod(w_shape)
    total_grid_elems = np.prod(g_shape)

    print("Self-Organizing Map (SOM) Summary")
    print("="*40)
    print(f"Map size:        {som.map_size}   (total neurons: {n_neurons})")
    print(f"Input dim:       {som.input_dim}")
    print(f"Learning rate:   {som.learning_rate}")
    print(f"Sigma (radius):  {som.sigma}")
    print()
    print("Structure:")
    print(f"  ├─ weights (Parameter)    shape={tuple(w_shape)}    params={total_weights}")
    print(f"  └─ grid    (Buffer)       shape={tuple(g_shape)}    elems={total_grid_elems}")
    print()
    print("Parameter counts:")
    print(f"  Total params:           {total_weights + total_grid_elems}")
    print(f"  Trainable params:       {total_weights}")
    print(f"  Non-trainable params:   {total_grid_elems}")

# Example usage:
som = SOM(map_size=(2, 5), input_dim=latent_dim, learning_rate=0.5)
som_detailed_summary(som)

Evaluate SOM
---

In [ ]:
# Predict cluster assignments for test latents
som_labels = som.predict(tf.convert_to_tensor(test_latents, dtype=tf.float32))

# Compute clustering metrics
sil_score_som = silhouette_score(test_latents, som_labels)
db_score_som = davies_bouldin_score(test_latents, som_labels)
ch_score_som = calinski_harabasz_score(test_latents, som_labels)

print("\nSOM Clustering Metrics:")
print(f"Silhouette Score:        {sil_score_som:.4f}")
print(f"Davies-Bouldin Index:    {db_score_som:.4f}")
print(f"Calinski-Harabasz Index: {ch_score_som:.4f}")
print(f"{'Quantization Error:':<25}{quant_error_som:.4f}")

Visualize SOM clustering
---

In [ ]:
# Run t-SNE on test latents
test_latents_2d = tsne.fit_transform(test_latents)

plt.figure(figsize=(10, 7))
sns.scatterplot(x=test_latents_2d[:, 0], y=test_latents_2d[:, 1], hue=som_labels, palette='tab10', s=15)
plt.title("t-SNE Visualization of SOM Clusters")
plt.show()
plt.savefig('som_clustering.png')
plt.close()

Comparing metric results of SHA-12 and SOM
---

In [ ]:
print("\nComparison Table:")
print(f"{'Metric':<35} {'DEC':<10} {'SOM':<10}")
print(f"{'Silhouette Score':<35} {sil_score:.4f}    {sil_score_som:.4f}")
print(f"{'Davies-Bouldin Index':<35} {db_score:.4f}    {db_score_som:.4f}")
print(f"{'Calinski-Harabasz Index':<35} {ch_score:.4f}    {ch_score_som:.4f}")
print(f"{'Final KL / Quantization Error':<35} {dec_val_kl:.4f}    {quant_error_som:.4f}")
